In [30]:
from codes.Env import Env
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
import numpy as np
import gym

In [31]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
env = Env('LunarLander-v2', "./gym-results/LunarLander/random")
_ = env.run(100, policy="random")
env.display_run()

In [32]:
env = gym.make('LunarLander-v2')
print(env.observation_space)
print(env.action_space)

Box(8,)
Discrete(4)


In [33]:
class deep_policy(object):
    def __init__(self, DeepQ):
        self.DeepQ = DeepQ
    
    def __call__(self, state):
        res = self.DeepQ.predict(np.array([state]))
        return np.argmax(res)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Activation

from keras.optimizers import SGD
from keras.models import clone_model
from keras.optimizers import adam

In [51]:
DeepQ = Sequential()

DeepQ.add(Dense(32))
DeepQ.add(Activation('relu'))
DeepQ.add(Dense(32))
DeepQ.add(Activation('relu'))
DeepQ.add(Dense(32))
DeepQ.add(Activation('relu'))
DeepQ.add(Dense(4))
DeepQ.add(Activation('linear'))

def huber_loss(y_true, y_pred):
        return tf.losses.huber_loss(y_true,y_pred)
    
opt = adam(lr=0.001)
Target_Q = clone_model(DeepQ)
DeepQ.compile(optimizer=opt, loss="mse", metrics=['mse'])

In [ ]:
MAX_MEMORY = 100000
EPISODES = 1000
BATCH_SIZE = 64
DISPLAY_EPISODES = 100
DISPLAY = True
C = 20
EPSILON = 1
EPSILON_DECAY = 0.99
MIN_EPSILON = 0.01
GAMMA = 0.99
MAX_STEP = 3000

In [ ]:
env = gym.make('LunarLander-v2')
epsilon = EPSILON
Memory = []
Target_Q.set_weights(DeepQ.get_weights())
BestQ.set_weights(DeepQ.get_weights())
best_score = 0
scores = []
for e in range(EPISODES):
    done = False
    state = env.reset()
    score = 0
    for step in range(MAX_STEP):
        # EPSILON GREEDY POLICY
        if np.random.uniform() < 1-epsilon:
            action = np.argmax(DeepQ.predict(np.array([state])))
        else:
            action = env.action_space.sample()
        # OBSERVATION
        new_state, reward, done, _ = env.step(action)
        score += reward
        # TRANSITION STRORING
        if len(Memory) > MAX_MEMORY:
            Memory = Memory[1:]
        Memory.append((state, action, reward, new_state, done))
        state = new_state
        # SAMPLE MINIBATCH
        batch = [Memory[_] for _ in np.random.randint(len(Memory), size=BATCH_SIZE).tolist()]
        batch_x, batch_a, batch_r, batch_x_new, batch_done = [], [], [], [], []
        for (x, a, r, x_new, x_done) in batch:
            batch_x.append(x)
            batch_a.append(a)
            batch_r.append(r)
            batch_x_new.append(x_new)
            batch_done.append(x_done)
        batch_x, batch_r, batch_x_new = np.array(batch_x), np.array(batch_r), np.array(batch_x_new)
        # BUILD TARGET
        y = DeepQ.predict(batch_x)
        future = Target_Q.predict(batch_x_new)
        for i, (y_, a_, x_done, r) in enumerate(zip(y, batch_a, batch_done, batch_r)):
            if x_done:
                y[i, a_] = r 
            else: 
                y[i, a_] = r + GAMMA*np.max(future[i,:])
        # PERFORM ONE UPDATE
        DeepQ.fit(batch_x, y, epochs=1, batch_size=BATCH_SIZE, verbose=0)
        if done:
            scores.append(score)
            epsilon = max(epsilon*EPSILON_DECAY, MIN_EPSILON)
            break
    # UPDATE THE TARGET NETWORK
    if (e+1) % C == 0:
        print("SCORE AFTER %i EPISODES: %.2f" % (e+1, np.mean(scores[-C:])))
        Target_Q.set_weights(DeepQ.get_weights())
        if np.mean(scores[-C:]) > scores:
            BestQ.set_weights(DeepQ.get_weights())
        
    if (e+1) % DISPLAY_EPISODES == 0 and DISPLAY:
        policy = deep_policy(DeepQ)
        test_env = Env('LunarLander-v2', "./gym-results/LunarLander/deep_train_%i" % (e+1))
        _ = test_env.run(500, policy=policy, **{})
        test_env.env.close()

In [ ]:
plt.plot([np.mean(scores[i-20:i]) for i in range(20, len(scores))])

In [ ]:
policy = deep_policy(DeepQ)
env = Env('LunarLander-v2', "./gym-results/LunarLander/deep")
_ = env.run(1000, policy=policy, **{})
env.display_run()

In [ ]:
future[i,:]

In [39]:
epsilon

0.19827425658891445

In [ ]:

test_env.env.close()